In [ ]:
# !pip install kfp==1.8.14

# create hello world component

In [1]:
from kfp.components import create_component_from_func

In [2]:
create_component_from_func??

Signature:
create_component_from_func(
    func: Callable,
    output_component_file: Union[str, NoneType] = None,
    base_image: Union[str, NoneType] = None,
    packages_to_install: List[str] = None,
    annotations: Union[Mapping[str, str], NoneType] = None,
)
Source:   
def create_component_from_func(
    func: Callable,
    output_component_file: Optional[str] = None,
    base_image: Optional[str] = None,
    packages_to_install: List[str] = None,
    annotations: Optional[Mapping[str, str]] = None,
):
    '''Converts a Python function to a component and returns a task factory
    (a function that accepts arguments and returns a task object).

    Args:
        func: The python function to convert
        base_image: Optional. Specify a custom Docker container image to use in the component. For lightweight components, the image needs to have python 3.5+. Default is the python image corresponding to the current python environment.
        output_component_file: Optional. Write a c

In [3]:
@create_component_from_func
def hello_world_component() -> str:
    ret = "Hello World!"
    print(ret)
    return ret

In [4]:
hello_world_component()

TaskSpec(component_ref=ComponentReference(name=None, digest=None, tag=None, url=None, spec=ComponentSpec(name='Hello world component', description=None, metadata=None, inputs=None, outputs=[OutputSpec(name='Output', type='String', description=None, annotations=None)], implementation=ContainerImplementation(container=ContainerSpec(image='python:3.7', command=['sh', '-ec', 'program_path=$(mktemp)\nprintf "%s" "$0" > "$program_path"\npython3 -u "$program_path" "$@"\n', 'def hello_world_component():\n    ret = "Hello World!"\n    print(ret)\n    return ret\n\ndef _serialize_str(str_value: str) -> str:\n    if not isinstance(str_value, str):\n        raise TypeError(\'Value "{}" has type "{}" instead of str.\'.format(\n            str(str_value), str(type(str_value))))\n    return str_value\n\nimport argparse\n_parser = argparse.ArgumentParser(prog=\'Hello world component\', description=\'\')\n_parser.add_argument("----output-paths", dest="_output_paths", type=str, nargs=1)\n_parsed_args = 

In [5]:
component_name = "0-1_hello-world-component.yaml"
hello_world_component.component_spec.save(component_name)

# create hello world pipeline

In [6]:
import kfp

In [7]:
@kfp.dsl.pipeline(name="hello_world_pipeline1", 
                  description="Hello World Pipeline!")
def hello_world_pipeline():
    hello_world_task = hello_world_component()

In [8]:
kfp.compiler.Compiler().compile(hello_world_pipeline, "hello-world-pipeline.zip")

component 생성후 pipeline 생성

In [ ]:
def hello_world_component2():
    ret = "Hello World!"
    print(ret)
    return ret

@kfp.dsl.pipeline(name="hello_world_pipeline2", description="Hello World Pipeline!")
def hello_world_pipeline():
    hello_world_op = kfp.components.func_to_container_op(hello_world_component2)
    hello_world_task = hello_world_op()

    kfp.compiler.Compiler().compile(hello_world_pipeline, "hello-world-pipeline2.zip")